In [108]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from string import punctuation
import os
from os import listdir
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize

In [106]:
def text_to_tokens(filename):
 '''Opens the input text file and
 returns a list of all of its words.'''
 file = open(filename, 'r')
 text = file.read()
 file.close()
 text = text.replace('\n', ' ')
 text = text.replace('  ', ' ')
 return text

In [6]:
cd ..

/Users/kamilapalys/Desktop/school/data450/capstone


In [7]:
pwd

'/Users/kamilapalys/Desktop/school/data450/capstone'

In [112]:
# example of using the function

filepath = 'data/text/cnn_trump.txt'
test_txt = text_to_tokens(filepath)
test_txt

"Donald Trump faces more than 30 counts related to business fraud in an indictment from a Manhattan grand jury, according to two sources familiar with the case – the first time in American history that a current or former president has faced criminal charges. Trump is expected to appear in court on Tuesday. The indictment has been filed under seal and will be announced in the coming days. The charges are not publicly known at this time. Manhattan District Attorney Alvin Bragg’s office has been investigating the former president in connection with his alleged role in a hush money payment scheme and cover-up involving adult film star Stormy Daniels that dates to the 2016 presidential election. Grand jury proceedings are secret, but a source familiar with the case told CNN that a witness gave about 30 minutes of testimony before it voted to indict Trump. The decision is sure to send shockwaves across the country, pushing the American political system – which has never seen one of its ex-l

In [114]:
def clean_text(text):
    '''Takes in a list of tokens and cleans them by converting
    to lowercase, removing punctuation, removing stopwords, 
    and stemming. Returns the new list of tokens.'''
    ps = PorterStemmer()
    # create list of stopwords 
    stopwords_list = stopwords.words('english')
    # make the text lowercase
    text = text.lower()
    # convert to ascii characters
    text = text.encode("ascii", "ignore").decode()
    new_text = ''
    for chr in text:
        # only keep characters in the string that are not punctuation symbols
        if chr in string.punctuation:
            text = text.replace(chr, ' ')
    text = text.replace('  ', ' ')
    # stem the tokens within the text
    tokens = text.split(" ")
    stemmed = []
    for token in tokens:
        stemmed_word = ps.stem(token)
        # only include new token in the cleaned list if not a stopword
        if stemmed_word not in stopwords_list:
            stemmed.append(stemmed_word)
    cleaned_text = " ".join(stemmed)
    return cleaned_text

In [115]:
cleaned_text = clean_text(test_txt)
cleaned_text

'donald trump face 30 count relat busi fraud indict manhattan grand juri accord two sourc familiar case first time american histori current former presid ha face crimin charg trump expect appear court tuesday indict ha file seal announc come day charg publicli known thi time manhattan district attorney alvin bragg offic ha investig former presid connect hi alleg role hush money payment scheme cover involv adult film star stormi daniel date 2016 presidenti elect grand juri proceed secret sourc familiar case told cnn wit gave 30 minut testimoni befor vote indict trump decis sure send shockwav across countri push american polit system ha never seen one ex leader confront crimin charg let alon run presid unchart water trump releas statement respons indict claim wa polit persecut elect interfer highest level histori believ thi witch hunt backfir massiv joe biden former presid said american peopl realiz exactli radic left democrat everyon see movement parti unit strong first defeat alvin bra

In [128]:
# how all the text files will be looped through

directory = 'data/text/'
file_list = []
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    file_list.append(f"{filepath}")
file_list
    # use preprocessing functions
    # somehow append each list of tokens (each article) to a dataframe
    

['data/text/nyp_affirmative.txt',
 'data/text/bbc_tanks.txt',
 'data/text/cnn_hamas.txt',
 'data/text/wsj_trump.txt',
 'data/text/wp_affirmative.txt',
 'data/text/bbc_balloon.txt',
 'data/text/wsj_affirmative.txt',
 'data/text/nyt_trump.txt',
 'data/text/wp_balloon.txt',
 'data/text/cnn_pentagon.txt',
 'data/text/cnn_tanks.txt',
 'data/text/bbc_hamas.txt',
 'data/text/nbc_tanks.txt',
 'data/text/nyp_hamas.txt',
 'data/text/.DS_Store',
 'data/text/cnn_balloon.txt',
 'data/text/fox_pentagon.txt',
 'data/text/wp_biden.txt',
 'data/text/nyp_tanks.txt',
 'data/text/nbc_hamas.txt',
 'data/text/abc_balloon.txt',
 'data/text/nyt_pentagon.txt',
 'data/text/fox_tanks.txt',
 'data/text/wp_trump.txt',
 'data/text/fox_hamas.txt',
 'data/text/nyp_balloon.txt',
 'data/text/abc_hamas.txt',
 'data/text/abc_santos.txt',
 'data/text/nyp_santos.txt',
 'data/text/fox_affirmative.txt',
 'data/text/wsj_biden.txt',
 'data/text/nyt_biden.txt',
 'data/text/abc_tanks.txt',
 'data/text/nbc_affirmative.txt',
 'dat

In [91]:
doc1 = 'start engine'
doc2 = 'start your engine'
doc3 = 'whenever youre ready to start'
docs = [doc1, doc2, doc3]

In [99]:
# takes all docs and makes them into a dataframe
# make sure that docs is just a list of all the tokens from all articles
# first make one list of tokens for each article and then add all those lists up
# mode is one of "binary", "count", "tfidf", "freq"
t = Tokenizer()

t.fit_on_texts(docs)
print(t)
encoded_docs = t.texts_to_matrix(docs, mode='binary')
#print(encoded_docs)
words = [x for x in t.word_index.keys()]
bow = pd.DataFrame(data = encoded_docs[:, 1:], columns=words)
bow['source'] = ['cnn']*2 + ['abc']
bow

,start,engine,your,whenever,youre,ready,to,source
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,cnn
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,cnn
2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,abc


In [118]:
t = Tokenizer()

docs = [cleaned_text]
t.fit_on_texts(docs)
print(t)
encoded_docs = t.texts_to_matrix(docs, mode='binary')
#print(encoded_docs)
words = [x for x in t.word_index.keys()]
bow = pd.DataFrame(data = encoded_docs[:, 1:], columns=words)
bow


,trump,indict,presid,ha,hi,former,wa,offic,charg,said,...,420,origin,liabil,reward,bonu,note,expens,intern,book,knowledg
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [74]:
from sklearn.feature_extraction.text import CountVectorizer

data = {'text': docs, 'source': ['cnn', 'abc', 'nbc']}
df = pd.DataFrame(data)

vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df['text'])
print(bow)

count_array = bow.toarray()
features = vectorizer.get_feature_names()
print(count_array)
print(features)
#df = pd.DataFrame(data=count_array, columns=features)

AttributeError: 'list' object has no attribute 'lower'